In [90]:
# Include the usual dependancies
using Pkg
Pkg.add("NBInclude")
using NBInclude
@nbinclude("Deps.ipynb")

# Activate dependancies being used in this day
using DelimitedFiles

 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


# --- Day 3: Crossed Wires ---
## --- Part One ---

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

`...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........`

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

`...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........`

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

* R75,D30,R83,U83,L12,D49,R71,U7,L72

  U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
  
  
* R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51

  U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?

In [235]:
# Parse file
input = readdlm("../inputs/daythree.txt", ',', String, '\n')
#input = ["R8" "U5" "L5" "D3"; "U7" "R6" "D4" "L4"]

# Initiate an empty array for parsing the input data
coords = Array{Array}(undef, size(input,1), size(input,2))

# Loop through each row (wire) then column (direction+distance)
for row in 1:size(input, 1)
    # Each row/wire has to start at the origin
    current = (x = 0, y =0)
    
    for col in 1:size(input, 2)
        if input[row, col][1] == 'R' # Right use case
            # Calculate new coordinates
            xval = current.x+parse(Int, input[row, col][2:end]) 
            yval = current.y
            
            # Create unit range from new coordinates and put into coords array
            if xval > current.x
                newcoord = [current.x:xval, current.y:yval]
                coords[row, col] = newcoord
            else
                newcoord = [xval:current.x, current.y:yval]
                coords[row, col] = newcoord
            end
            
            # Update new position
            current = (x = xval, y = yval)
        elseif input[row, col][1] == 'L' # Left use case
            # Calculate new coordinates
            xval = current.x-parse(Int, input[row, col][2:end])
            yval = current.y
            
            # Create unit range from new coordinates and put into coords array
            if xval > current.x
                newcoord = [current.x:xval, current.y:yval]
                coords[row, col] = newcoord
            else
                newcoord = [xval:current.x, current.y:yval]
                coords[row, col] = newcoord
            end
            
            # Update new position
            current = (x = xval, y = yval)
        elseif input[row, col][1] == 'U' # Up use case
            # Calculate new coordinates
            xval = current.x
            yval = current.y+parse(Int, input[row, col][2:end])
            
            # Create unit range from new coordinates and put into coords array
            if yval > current.y
                newcoord = [current.x:xval, current.y:yval]
                coords[row, col] = newcoord
            else
                newcoord = [xval:current.x, yval:current.y]
                coords[row, col] = newcoord
            end
            
            # Update new position
            current = (x = xval, y = yval)
        elseif input[row, col][1] == 'D' # Down use case
            # Calculate new coordinates
            xval = current.x
            yval = current.y-parse(Int, input[row, col][2:end])
            
            # Create unit range from new coordinates and put into coords array
            if yval > current.y
                newcoord = [current.x:xval, current.y:yval]
                coords[row, col] = newcoord
            else
                newcoord = [xval:current.x, yval:current.y]
                coords[row, col] = newcoord
            end
            
            # Update new position
            current = (x = xval, y = yval)
        end
    end 
end

# Define clostest value to zero as infinite
closest = Inf
# Loop through each wires coords
for (index1, set1) in enumerate(coords[1, :]), (index2, set2) in enumerate(coords[2, :])
    # Find the x and y intersects of both the wires coords
    x = intersect(collect(set1[1]), collect(set2[1]))
    y = intersect(collect(set1[2]), collect(set2[2]))

    # Check for zeros/failed intersects
    if isempty(x) == false && isempty(y) == false && x != [0] && y != [0]
        # Check if intersect is closer to zero than previous intersect and update it
        if abs(x[1])+abs(y[1]) < abs(closest)
            closest = abs(x[1])+abs(y[1]);
        end
    end
end

# Print result
println(closest)

1112


## --- Part Two ---

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

`...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........`
    
In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

`R75,D30,R83,U83,L12,D49,R71,U7,L72
U62,R66,U55,R34,D71,R55,D58,R83` = 610 steps

`R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
U98,R91,D20,R16,D67,R40,U7,R15,U6,R7` = 410 steps
            
What is the fewest combined steps the wires must take to reach an intersection?

In [247]:
# Parse file
input = readdlm("../inputs/daythree.txt", ',', String, '\n')
#input = ["R8" "U5" "L5" "D3"; "U7" "R6" "D4" "L4"]

# Initiate an empty array for parsing the input data
coords = Array{Array}(undef, size(input,1), size(input,2))

# Loop through each row (wire) then column (direction+distance)
for row in 1:size(input, 1)
    # Each row/wire has to start at the origin
    current = (x = 0, y =0)
    for col in 1:size(input, 2)
            if input[row, col][1] == 'R' # Right use case
                # Calculate new coordinates
                xval = current.x+parse(Int, input[row, col][2:end]) 
                yval = current.y

                # Create unit range from new coordinates and put into coords array
                if xval > current.x
                    newcoord = [current.x:xval, current.y:yval]
                    coords[row, col] = newcoord
                else
                    newcoord = [xval:current.x, current.y:yval]
                    coords[row, col] = newcoord
                end

                # Update new position
                current = (x = xval, y = yval)
            elseif input[row, col][1] == 'L' # Left use case
                # Calculate new coordinates
                xval = current.x-parse(Int, input[row, col][2:end])
                yval = current.y

                # Create unit range from new coordinates and put into coords array
                if xval > current.x
                    newcoord = [current.x:xval, current.y:yval]
                    coords[row, col] = newcoord
                else
                    newcoord = [xval:current.x, current.y:yval]
                    coords[row, col] = newcoord
                end

                # Update new position
                current = (x = xval, y = yval)
            elseif input[row, col][1] == 'U' # Up use case
                # Calculate new coordinates
                xval = current.x
                yval = current.y+parse(Int, input[row, col][2:end])

                # Create unit range from new coordinates and put into coords array
                if yval > current.y
                    newcoord = [current.x:xval, current.y:yval]
                    coords[row, col] = newcoord
                else
                    newcoord = [xval:current.x, yval:current.y]
                    coords[row, col] = newcoord
                end

                # Update new position
                current = (x = xval, y = yval)
            elseif input[row, col][1] == 'D' # Down use case
                # Calculate new coordinates
                xval = current.x
                yval = current.y-parse(Int, input[row, col][2:end])

                # Create unit range from new coordinates and put into coords array
                if yval > current.y
                    newcoord = [current.x:xval, current.y:yval]
                    coords[row, col] = newcoord
                else
                    newcoord = [xval:current.x, yval:current.y]
                    coords[row, col] = newcoord
                end

                # Update new position
                current = (x = xval, y = yval)
            end
    end 
end

# Initialise least amount of steps to infinite
closest = Inf
# Loop through each wires coords
for (index1, set1) in enumerate(coords[1, :]), (index2, set2) in enumerate(coords[2, :])
    # Find the x and y intersects of both the wires coords
    x = intersect(collect(set1[1]), collect(set2[1]))
    y = intersect(collect(set1[2]), collect(set2[2]))

    # Check for zeros/failed intersects
    if isempty(x) == false && isempty(y) == false && x != [0] && y != [0]
        # Initialise step counter to 0
        count = 0
        
        # Loop through all of wire1's coords
        for i in collect(1:index1)  
            if i == index1 # Check if loop is where intersect is
                # Check (x) for previous fixed coord (same number->same number)
                if collect(coords[1, i-1][1])[end] == collect(coords[1, i-1][1])[1]
                    # Minus intersect from coord
                    count += abs(x[1]-collect(coords[1, i-1][1])[end])
                end
                
                # Check (y) for previous fixed coord (same number->same number)
                if collect(coords[1, i-1][2])[end] == collect(coords[1, i-1][2])[1]
                    # Minus intersect from coord
                    count += abs(y[1]-collect(coords[1, i-1][2])[end])
                end
            else
                # End of line - start of line coord add to count
                count += collect(coords[1, i][1])[end]-collect(coords[1, i][1])[1]
                count += collect(coords[1, i][2])[end]-collect(coords[1, i][2])[1]
            end
        end
        
        # Loop through all of wire1's coords
        for j in collect(1:index2)
            if j == index2  # Check if loop is where intersect is
                # Check (x) for previous fixed coord (same number->same number)
                if collect(coords[2, j-1][1])[end] == collect(coords[2, j-1][1])[1]
                    # Minus intersect from coord
                    count += abs(x[1]-collect(coords[2, j-1][1])[end])
                end
                
                # Check (y) for previous fixed coord (same number->same number)
                if collect(coords[2, j-1][2])[end] == collect(coords[2, j-1][2])[1]
                    # Minus intersect from coord
                    count += abs(y[1]-collect(coords[2, j-1][2])[end])
                end
            else 
                # End of line - start of line coord add to count
                count += collect(coords[2, j][1])[end]-collect(coords[2, j][1])[1]
                count += collect(coords[2, j][2])[end]-collect(coords[2, j][2])[1]
            end
        end
        
        # Update shortest count
        if abs(count) < abs(closest)
            closest = abs(count)
        end
    end
end
println(closest)

45560
